#**Covid 19 detection on image dataset**

In [ ]:
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

import tensorflow as tf

import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATASET_DIR = "/content/drive/MyDrive/data_upload_v3/Dataset_new"

In [ ]:
os.listdir(DATASET_DIR)

['Covid', 'Non-Covid']

In [ ]:
from numpy import inf
import numpy as npy
from timeit import default_timer as timer
import numpy as np

import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
def InsideCircle(xc, yc, R): # returns list of [x,y] points inside a circle with center (xc,yc) and radius R
    L = []
    for i in range(2*R+1):  
        for j in range(2*R+1): 
            xx = xc-R+j
            yy = yc-R+i
            ll = (R-j)*(R-j)+(R-i)*(R-i)
            if (ll <= R*R):
                q = [xx,yy]
                L.append(q) 

    return L

def InsideCircleX(xc, yc, R): # returns list of [x] points inside a circle with center (xc,yc) and radius R
    Lx = []
    for i in range(2*R+1):  
        for j in range(2*R+1): 
            xx = xc-R+j
            ll = (R-j)*(R-j)+(R-i)*(R-i)
            if (ll <= R*R):
                Lx.append(xx)

    return Lx

def InsideCircleY(xc, yc, R): # returns list of [y] points inside a circle with center (xc,yc) and radius R
    Ly = []
    for i in range(2*R+1):  
        for j in range(2*R+1): 
            yy = yc-R+i
            ll = (R-j)*(R-j)+(R-i)*(R-i)
            if (ll <= R*R):
                Ly.append(yy)
    return Ly

def DistanceMatrix_cpu(boundary_x, boundary_y, internal_points_x, internal_points_y):
    dist = []
    dist_x = (boundary_x[:,npy.newaxis] - internal_points_x[:])**2

    dist_y = (boundary_y[:,npy.newaxis] - internal_points_y[:])**2
    #print(boundary_x)
    #print(internal_points_x)
    #print(internal_points_x)
    #print("Dist_X")
    #print(dist_x.shape)
    #print("Dist_Y")
    #print(dist_y.shape)
    #print(dist_x)
   
    return np.sqrt(dist_x+dist_y)

def normalize(array):
    return (array - array.mean()) / array.std()

In [ ]:
#initialize object that keeps track of our boundary values
dataset = {
    'n' :[],
    'c' : []
}

#**Resizing and loading the normal and covid images.**

In [ ]:
from PIL import Image
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.144])
normal_images = []
for img_path in glob.glob(DATASET_DIR + '/Non-Covid/*'):
  f_img=img_path
  img = Image.open(img_path)
  img = img.resize((224,224))
  img.save(f_img)
  
  
for img_path in glob.glob(DATASET_DIR + '/Non-Covid/*'):
    img = mpimg.imread(img_path)
    if len(img.shape) > 2:
        img = rgb2gray(mpimg.imread(img_path))
        
    normal_images.append(img)
    print(img)

Streaming output truncated to the last 5000 lines.
 [105 113 107 ...  39  51  62]
 [ 92 101  98 ...  38  50  63]]
[[ 3  2  1 ...  8  8  8]
 [ 1  1  2 ...  8  8  8]
 [ 0  0  0 ...  8  8  8]
 ...
 [12  9 14 ...  8  8  7]
 [ 9  8 15 ...  9  7  6]
 [ 8 12 25 ...  9  7  6]]
[[ 28  28  28 ... 252 255 255]
 [ 76  76  76 ... 248 255 255]
 [ 42  42  42 ... 244 255 250]
 ...
 [  1   2   3 ...   1   0   0]
 [  1   2   3 ...   1   0   0]
 [  1   2   3 ...   1   0   1]]
[[  9   8   7 ...  35 132 166]
 [  8   8   7 ...  21 117 166]
 [  7   6   5 ...  16 107 153]
 ...
 [ 14  14  14 ...  30  55  92]
 [ 14  14  14 ...  38  59  92]
 [ 14  14  14 ...  43  58  84]]
[[  0   5   8 ...  23  23   8]
 [  0   5   8 ...  22  23  17]
 [  0   5   8 ...  24  20  22]
 ...
 [ 10 107 143 ...  43  47  42]
 [ 16  73  85 ...  38  40  40]
 [ 14  18   6 ...  13  11  13]]
[[4 4 4 ... 1 0 0]
 [3 3 3 ... 2 3 3]
 [2 2 2 ... 0 2 3]
 ...
 [4 2 2 ... 2 1 0]
 [4 2 2 ... 3 1 0]
 [4 2 2 ... 3 1 0]]
[[ 28  37  42 ... 135 125  90]
 [ 

In [ ]:
len(normal_images)

5000

In [ ]:
normal_images[0]

array([[ 24,  30,  28, ...,  14,  13,  14],
       [ 29,  20,  19, ...,  16,  12,  10],
       [ 26,  19,  18, ...,  20,  14,   8],
       ...,
       [  0,   0,   7, ...,   4,   0,   0],
       [  8,   0,   4, ...,   0,   0,  14],
       [171,  49,   0, ...,   0,  21,  82]], dtype=uint8)

In [ ]:


dataset['n'] = []
for img in normal_images:
  Rad_Normal = 40 #13 #4 #10 #100
  Delta_Normal = 5 #40
  
  #print("XRay grayscale intensity for normal_case[0]")
  #print(img.shape)

    # Apply a circular probe (circle1) to the Normal Image
    # with (Center_Normal_X, Center_Normal_Y) and radius Rad_Normal
    # A concentric circumference (circle1_1) with Rad_Normal + Delta_Normal
  Rad_Normal = 40 #13 #4 #10 #100
  Delta_Normal = 5 #40
  for _ in range(10):
    Center_Normal_X, Center_Normal_Y = int(img.shape[0])//2 , int(img.shape[0] *.75) //2
    Center_Normal_X += random.randint(50, int((img.shape[0]-Center_Normal_X) *.6))
    Center_Normal_Y += random.randint(50, int((img.shape[1]-Center_Normal_Y) *.6))
    #print('center: ({}, {})'.format(Center_Normal_X, Center_Normal_Y))
    # Find points on the boundary
    Boundary_Points = 1000 #100 #10 #1000
    R = Rad_Normal + Delta_Normal
    Center_x = Center_Normal_X
    Center_y = Center_Normal_Y
    Circ_Bound = np.linspace(0, 2*np.pi, Boundary_Points); 
    #print(Circ_Bound)
    Circ_Bound_x = R * np.cos(Circ_Bound) + Center_x
    Circ_Bound_y = R * np.sin(Circ_Bound) + Center_y    
    #print(Circ_Bound_x)
    #print(Circ_Bound_y)
    #Find all points within circular probe
    #In_Circle = InsideCircle(Center_x, Center_y, Rad_Normal) # Returns (x,y) vector
    In_CircleX = InsideCircleX(Center_x, Center_y, Rad_Normal) # Returns X-components
    In_CircleY = InsideCircleY(Center_x, Center_y, Rad_Normal) # Returns Y components
    #print("In_CircleX")
    #print(In_CircleX)
    #print("In_CircleY")
    #print(In_CircleY)
    #print("Circ_Bound_x (in this example 10 points on the boundary)")
    #print(Circ_Bound_x)
    # Distance matrix between all points in the circular probe and the external circular boundary
    DM_Distance = DistanceMatrix_cpu((Circ_Bound_x), (Circ_Bound_y),(In_CircleX), (In_CircleY));
    #print("Distance Matrix (in this example 10 by 49)")
    #print(DM_Distance)
    Kernel1 = 1./DM_Distance
    #print("Kernel1 (Inverse Radial Function)")
    #print(Kernel1)
    #Create ImageSample
    ImageSample = []
    PointsInside = len(In_CircleX)
    #print("PointsInside")
    #print(PointsInside)
    for i in range(PointsInside):
        x = In_CircleX[i]
        y = In_CircleY[i]
        ImageSample.append(img[x,y])
    # print(ImageSample)
    BoundaryFunction = normalize(np.dot(Kernel1, ImageSample))
    #print(BoundaryFunction)

    dataset['n'].append(BoundaryFunction)
    


In [ ]:
covid_images = []
for img_path in glob.glob(DATASET_DIR + '/Covid/*'):
  f_img=img_path
  img = Image.open(img_path)
  img = img.resize((224,224))
  img.save(f_img)
for img_path in glob.glob(DATASET_DIR + '/Covid/*'):
    img = mpimg.imread(img_path)
    if len(img.shape) > 2:
        img = rgb2gray(mpimg.imread(img_path))
    covid_images.append(img)

In [ ]:
len(covid_images)

184

In [ ]:
covid_images[0]

array([[0.69878433, 0.67050983, 0.65491767, ..., 0.81244707, 0.60180003,
        0.49156863],
       [0.65435296, 0.62203924, 0.60992159, ..., 0.97054118, 0.85913727,
        0.71889414],
       [0.63298434, 0.60874904, 0.59780395, ..., 1.0161451 , 1.01901177,
        1.00459216],
       ...,
       [0.90421962, 0.90421962, 0.90135295, ..., 0.86439217, 0.8724706 ,
        0.86035295],
       [0.90825883, 0.90248236, 0.9001804 , ..., 0.86439217, 0.86035295,
        0.85227452],
       [0.90074511, 0.89614118, 0.89210197, ..., 0.85631374, 0.86035295,
        0.85631374]])

In [ ]:
import random
dataset['c'] = []
for img in covid_images:

    #print("XRay grayscale intensity for normal_case[0]")
    #print(img.shape)

    # Apply a circular probe (circle1) to the Normal Image
    # with (Center_Normal_X, Center_Normal_Y) and radius Rad_Normal
    # A concentric circumference (circle1_1) with Rad_Normal + Delta_Normal 
    # is applied as a measurement boundary (Delta_Normal: a gap between probe and boundary)

    #Center_Normal_X, Center_Normal_Y = 1300, 800 # 100, 100
    Center_Normal_X, Center_Normal_Y = int(img.shape[0])//2 , int(img.shape[0] *.75) //2 # 100, 100????????
    Rad_Normal = 40 #13 #4 #10 #100
    Delta_Normal = 5 #40
    for _ in range(10):
        Center_Normal_X, Center_Normal_Y = int(img.shape[0])//2 , int(img.shape[0] *.75) //2 # 100, 100????????

        ## Randomly select an x/y point near the center of the circle
        Center_Normal_X += random.randint(50, int((img.shape[0]-Center_Normal_X) *.6))
        Center_Normal_Y += random.randint(50, int((img.shape[1]-Center_Normal_Y) *.6))
        #print('center: ({}, {})'.format(Center_Normal_X, Center_Normal_Y))
        # Find points on the boundary
        Boundary_Points = 1000 #100 #10 #1000
        R = Rad_Normal + Delta_Normal
        Center_x = Center_Normal_X
        Center_y = Center_Normal_Y
        Circ_Bound = np.linspace(0, 2*np.pi, Boundary_Points); 
        #print(Circ_Bound)
        Circ_Bound_x = R * np.cos(Circ_Bound) + Center_x
        Circ_Bound_y = R * np.sin(Circ_Bound) + Center_y    
        #print(Circ_Bound_x)
        #print(Circ_Bound_y)
        #Find all points within circular probe
        #In_Circle = InsideCircle(Center_x, Center_y, Rad_Normal) # Returns (x,y) vector
        In_CircleX = InsideCircleX(Center_x, Center_y, Rad_Normal) # Returns X-components
        In_CircleY = InsideCircleY(Center_x, Center_y, Rad_Normal) # Returns Y components
        #print("In_CircleX")
        #print(In_CircleX)
        #print("In_CircleY")
        #print(In_CircleY)
        #print("Circ_Bound_x (in this example 10 points on the boundary)")
        #print(Circ_Bound_x)

        # Distance matrix between all points in the circular probe and the external circular boundary
        DM_Distance = DistanceMatrix_cpu((Circ_Bound_x), (Circ_Bound_y),(In_CircleX), (In_CircleY));
        #print("Distance Matrix (in this example 10 by 49)")
        #print(DM_Distance)
        Kernel1 = 1./DM_Distance
        #print("Kernel1 (Inverse Radial Function)")
        #print(Kernel1)
        #Create ImageSample
        ImageSample = []
        PointsInside = len(In_CircleX)
        #print("PointsInside")
        #print(PointsInside)
        for i in range(PointsInside):
            x = In_CircleX[i]
            y = In_CircleY[i]
            ImageSample.append(img[x,y])
    #     print(ImageSample)

        #BoundaryFunction_unnorm = np.dot(Kernel1, ImageSample)
        #print(BoundaryFunction_unnorm)
        #plt.suptitle("Boundary Function - before normalization")
        #plt.plot(BoundaryFunction_unnorm)
        #plt.ylabel('Boundary Value')
        #plt.show()


        BoundaryFunction = normalize(np.dot(Kernel1, ImageSample))
        dataset['c'].append(BoundaryFunction)



#**Preprocess data (data transformation experiments)**

In [ ]:
'''
At this stage, I like to keep the original dataset intact and experiment 
with different transformations by copying the values over to a new object
'''
data_modified = {
    'n' : [],
    'c' : [],
}


import copy
for shape in dataset:
    
    ##Normalize each class by L2 norm
    t = copy.deepcopy(dataset[shape])
    t/=np.linalg.norm(t)
#     t -=np.mean(t)
#     t/=np.std(t)
    for item in t:
        data_modified[shape].append(item)
        

#Visualize Boundary Values

In [ ]:
# for _ in range(5):
#     idx = random.randint(0, 139)
#     bf = data_modified['n'][idx]
#     bfc = data_modified['c'][idx]

#     plt.plot(bf, label='normal')
#     plt.plot(bfc, label='covid')
#     plt.legend()
#     plt.show()

#**Create Training and Test Datasets**

In [ ]:
train_x = []
test_x = []
train_y=[]
test_y=[]



i = 0
max_len = []
for s in data_modified:
    max_len.append(len(data_modified[s]))
    
max_len = np.min(max_len)
for shape in data_modified:
    #select only the first 140 elements to make both labels have same number of objects
    data = data_modified[shape][:max_len]

    if len(data) == 0:
        continue
        
    print('shape', shape, len(data))

    data = np.asarray(data)
    print(data.shape)
#     data = data.reshape(data.shape[0], data.shape[1], 1)
#     data /= np.linalg.norm(data)
    # data = data[:,0]
    #data = np.abs(np.apply_along_axis(np.fft.fft, 1, data))
    random_range = np.arange(data.shape[0])
    np.random.shuffle(random_range)
    train_range = int(random_range.shape[0] *.80)#80 : 20 ratio division
    if i== 0:
        train_y = [i] * data[random_range[:train_range]].shape[0]
        test_y = [i] * data[random_range[train_range:]].shape[0]
        train_x = data[random_range[:train_range]]
        test_x = data[random_range[train_range:]]
    else:
        train_y = np.concatenate((train_y, [i] * data[random_range[:train_range]].shape[0]), axis=0)
        test_y = np.concatenate((test_y, ([i] * data[random_range[train_range:]].shape[0])), axis=0)
        train_x = np.concatenate((train_x, data[random_range[:train_range]]), axis=0)
        test_x = np.concatenate((test_x, data[random_range[train_range:]]), axis=0)
    i+=1


shape n 1840
(1840, 1000)
shape c 1840
(1840, 1000)


In [ ]:
train_x.shape

(2944, 1000)

In [ ]:
test_x.shape

(736, 1000)

#**All scores Function**

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import matthews_corrcoef,cohen_kappa_score,roc_auc_score
from sklearn.metrics import accuracy_score
def all_metrices(y_test,y_pred):
  acc=accuracy_score(y_test, y_pred)
  auc=roc_auc_score(y_test,y_pred)
  precision=precision_score(y_test, y_pred)
  recall=recall_score(y_test, y_pred)
  f1=f1_score(y_test, y_pred)
  mcc=matthews_corrcoef(y_test,y_pred)
  kappa=cohen_kappa_score(y_test,y_pred)

  print('Accuracy: %.2f' % (acc*100))
  print('Roc_Auc Score: %.2f' % (auc*100))
  print('Precision: %.2f' % (precision*100))
  print('recall: %.2f' % (recall*100))
  print('fl score: %.2f' % (f1*100))
  print('Mcc Score: %.2f' % (mcc))
  print('Kappa score: %.2f' % (kappa))
  print('\n')

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        all_metrices(y_train,pred)
    elif train==False:
        pred = clf.predict(X_test)
        
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")
        all_metrices(y_test,pred)

#Models Without Cross Validation

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
knn_clf.fit(train_x, train_y)

print_score(knn_clf, train_x, train_y, test_x, test_y, train=True)
print_score(knn_clf, train_x, train_y, test_x, test_y, train=False)

Train Result:
Accuracy Score: 99.97%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.999321     1.000000   0.99966     0.999661      0.999661
recall        1.000000     0.999321   0.99966     0.999660      0.999660
f1-score      0.999660     0.999660   0.99966     0.999660      0.999660
support    1472.000000  1472.000000   0.99966  2944.000000   2944.000000
_______________________________________________
Confusion Matrix: 
 [[1472    0]
 [   1 1471]]

Accuracy: 99.97
Roc_Auc Score: 99.97
Precision: 100.00
recall: 99.93
fl score: 99.97
Mcc Score: 1.00
Kappa score: 1.00


Test Result:
Accuracy Score: 99.86%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy   macro avg  weighted avg
precision    0.997290    1.000000  0.998641    0.998645      0.998645
recall       1.000000    0.997283  0.998641    0.998641      0.

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(train_x, train_y)
print_score(lr_clf, train_x, train_y, test_x, test_y, train=True)
print_score(lr_clf, train_x, train_y, test_x, test_y, train=False)


Train Result:
Accuracy Score: 97.32%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.949065     1.000000  0.973166     0.974533      0.974533
recall        1.000000     0.946332  0.973166     0.973166      0.973166
f1-score      0.973867     0.972426  0.973166     0.973146      0.973146
support    1472.000000  1472.000000  0.973166  2944.000000   2944.000000
_______________________________________________
Confusion Matrix: 
 [[1472    0]
 [  79 1393]]

Accuracy: 97.32
Roc_Auc Score: 97.32
Precision: 100.00
recall: 94.63
fl score: 97.24
Mcc Score: 0.95
Kappa score: 0.95


Test Result:
Accuracy Score: 97.01%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy   macro avg  weighted avg
precision    0.943590    1.000000  0.970109    0.971795      0.971795
recall       1.000000    0.940217  0.970109    0.970109      0.

In [ ]:

from sklearn.svm import SVC


svm_clf = SVC(kernel='rbf', gamma=0.1, C=1.0)
svm_clf.fit(train_x, train_y)

print_score(svm_clf, train_x, train_y, test_x, test_y, train=True)
print_score(svm_clf, train_x, train_y, test_x, test_y, train=False)

Train Result:
Accuracy Score: 97.69%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.955844     1.000000  0.976902     0.977922      0.977922
recall        1.000000     0.953804  0.976902     0.976902      0.976902
f1-score      0.977424     0.976356  0.976902     0.976890      0.976890
support    1472.000000  1472.000000  0.976902  2944.000000   2944.000000
_______________________________________________
Confusion Matrix: 
 [[1472    0]
 [  68 1404]]

Accuracy: 97.69
Roc_Auc Score: 97.69
Precision: 100.00
recall: 95.38
fl score: 97.64
Mcc Score: 0.95
Kappa score: 0.95


Test Result:
Accuracy Score: 97.69%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy   macro avg  weighted avg
precision    0.955844    1.000000  0.976902    0.977922      0.977922
recall       1.000000    0.953804  0.976902    0.976902      0.

In [ ]:
from sklearn.tree import DecisionTreeClassifier


tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(train_x, train_y)
print_score(tree_clf, train_x, train_y, test_x, test_y, train=True)
print_score(tree_clf, train_x, train_y, test_x, test_y, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
                0       1  accuracy  macro avg  weighted avg
precision     1.0     1.0       1.0        1.0           1.0
recall        1.0     1.0       1.0        1.0           1.0
f1-score      1.0     1.0       1.0        1.0           1.0
support    1472.0  1472.0       1.0     2944.0        2944.0
_______________________________________________
Confusion Matrix: 
 [[1472    0]
 [   0 1472]]

Accuracy: 100.00
Roc_Auc Score: 100.00
Precision: 100.00
recall: 100.00
fl score: 100.00
Mcc Score: 1.00
Kappa score: 1.00


Test Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
               0      1  accuracy  macro avg  weighted avg
precision    1.0    1.0       1.0        1.0           1.0
recall       1.0    1.0       1.0        1.0           1.0
f1-score     1.0    1.0       1.0        1.0           1.0
support    368.0  368

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

rf_clf = RandomForestClassifier(n_estimators=2000, random_state=42)
rf_clf.fit(train_x, train_y)
print_score(rf_clf, train_x, train_y, test_x, test_y, train=True)
print_score(rf_clf, train_x, train_y, test_x, test_y, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
                0       1  accuracy  macro avg  weighted avg
precision     1.0     1.0       1.0        1.0           1.0
recall        1.0     1.0       1.0        1.0           1.0
f1-score      1.0     1.0       1.0        1.0           1.0
support    1472.0  1472.0       1.0     2944.0        2944.0
_______________________________________________
Confusion Matrix: 
 [[1472    0]
 [   0 1472]]

Accuracy: 100.00
Roc_Auc Score: 100.00
Precision: 100.00
recall: 100.00
fl score: 100.00
Mcc Score: 1.00
Kappa score: 1.00


Test Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
               0      1  accuracy  macro avg  weighted avg
precision    1.0    1.0       1.0        1.0           1.0
recall       1.0    1.0       1.0        1.0           1.0
f1-score     1.0    1.0       1.0        1.0           1.0
support    368.0  368

In [ ]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier()
xgb_clf.fit(train_x, train_y)
print_score(xgb_clf, train_x, train_y, test_x, test_y, train=True)
print_score(xgb_clf, train_x, train_y, test_x, test_y, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
                0       1  accuracy  macro avg  weighted avg
precision     1.0     1.0       1.0        1.0           1.0
recall        1.0     1.0       1.0        1.0           1.0
f1-score      1.0     1.0       1.0        1.0           1.0
support    1472.0  1472.0       1.0     2944.0        2944.0
_______________________________________________
Confusion Matrix: 
 [[1472    0]
 [   0 1472]]

Accuracy: 100.00
Roc_Auc Score: 100.00
Precision: 100.00
recall: 100.00
fl score: 100.00
Mcc Score: 1.00
Kappa score: 1.00


Test Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
               0      1  accuracy  macro avg  weighted avg
precision    1.0    1.0       1.0        1.0           1.0
recall       1.0    1.0       1.0        1.0           1.0
f1-score     1.0    1.0       1.0        1.0           1.0
support    368.0  368

#Models with cross Validation

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np
params = {"C": [0.05179474679231213]
          }

lr_clf = LogisticRegression()

lr_cv = GridSearchCV(lr_clf, params, scoring="accuracy", n_jobs=-1, verbose=1, cv=5)
lr_cv.fit(train_x, train_y)#Dummy Variables splitted dataset
best_params = lr_cv.best_params_
print(f"Best parameters: {best_params}")
lr_clf = LogisticRegression(**best_params)

lr_clf.fit(train_x, train_y)
print_score(lr_clf, train_x, train_y, test_x, test_y, train=True)
print_score(lr_clf, train_x, train_y, test_x, test_y, train=False)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters: {'C': 0.05179474679231213}
Train Result:
Accuracy Score: 97.32%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.949065     1.000000  0.973166     0.974533      0.974533
recall        1.000000     0.946332  0.973166     0.973166      0.973166
f1-score      0.973867     0.972426  0.973166     0.973146      0.973146
support    1472.000000  1472.000000  0.973166  2944.000000   2944.000000
_______________________________________________
Confusion Matrix: 
 [[1472    0]
 [  79 1393]]

Accuracy: 97.32
Roc_Auc Score: 97.32
Precision: 100.00
recall: 94.63
fl score: 97.24
Mcc Score: 0.95
Kappa score: 0.95


Test Result:
Accuracy Score: 97.01%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy   macro avg  weighted avg
precision    0.943590    1.000

In [ ]:
from sklearn.utils.fixes import scipy
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, cv=5,refit = True, verbose = 3)
grid.fit(train_x, train_y)

print_score(grid,train_x, train_y, test_x, test_y, train=True)
print_score(grid, train_x, train_y, test_x, test_y, train=False)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.499 total time=   3.2s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.499 total time=   3.1s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.499 total time=   3.1s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.499 total time=   3.1s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.978 total time=   3.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.499 total time=   3.1s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.499 total time=   3.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.499 total time=   3.1s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.499 total time=   3.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.978 total time=   3.1s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.499 total time=   3.1s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

In [ ]:
dt = DecisionTreeClassifier(random_state=42)
params = {
    'min_impurity_decrease':[0.1],
    'max_depth':[2],
    'min_samples_leaf':[27],
    'criterion': [ "entropy"]
}
grid_search = GridSearchCV(estimator=dt, 
                           param_grid=params, 
                           cv=5, n_jobs=-1 ,verbose=1,scoring = "accuracy")
grid_search.fit(train_x, train_y)
print_score(grid_search, train_x, train_y, test_x, test_y, train=True)
print_score(grid_search, train_x, train_y, test_x, test_y, train=False)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Train Result:
Accuracy Score: 99.42%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.989240     0.999314  0.994226     0.994277      0.994277
recall        0.999321     0.989130  0.994226     0.994226      0.994226
f1-score      0.994255     0.994196  0.994226     0.994225      0.994225
support    1472.000000  1472.000000  0.994226  2944.000000   2944.000000
_______________________________________________
Confusion Matrix: 
 [[1471    1]
 [  16 1456]]

Accuracy: 99.42
Roc_Auc Score: 99.42
Precision: 99.93
recall: 98.91
fl score: 99.42
Mcc Score: 0.99
Kappa score: 0.99


Test Result:
Accuracy Score: 99.32%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy   macro avg  weighted avg
precision    0.989218    0.997260  0.993207    0.993239      0.993239
recal